<a href="https://colab.research.google.com/github/vijayamurugan27/GenerativeAI/blob/main/Advanced_RAG_based_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modern AI Pro: Advanced Chatbot on your enterprise content

We will scrape the web content as well as our PDFs to answer user queries more effectively.

In [1]:
!pip install -U -q langchain langchain_groq langchain-community chromadb pypdf2 sentence-transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 7.7 MB/s e

In [2]:
from langchain.schema.document import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from PyPDF2 import PdfReader

In [3]:
# 1. Get and Parse PDF
!gdown 1I8Ga_BK23OA8EEv3DtAmo5zw_Quud0ew
reader = PdfReader('arso.pdf')
text = ""
for i in range(0, len(reader.pages)):
    page = reader.pages[i]
    text += page.extract_text() + " "

Downloading...
From: https://drive.google.com/uc?id=1I8Ga_BK23OA8EEv3DtAmo5zw_Quud0ew
To: /content/arso.pdf
100% 2.90M/2.90M [00:00<00:00, 157MB/s]


In [4]:
#2. Get the embeddings.
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})

<ipython-input-4-cc931b3ca3cf>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optio

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# 3. Split document & store it in db
documents = [Document(page_content=text, metadata={"source": "local"})]
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=40)
all_splits = text_splitter.split_documents(documents)

collection = Chroma.from_documents(
    documents=all_splits, embedding=embeddings, persist_directory="doc_vectors")


In [6]:
# 4. Add more items to the collection
loader = WebBaseLoader(web_path=("https://mitrarobot.com/"))
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=40)
splits = text_splitter.split_documents(docs)

for s in splits:
    collection.add_documents([s])

In [7]:
# 5. Setup the LLM
from langchain_groq import ChatGroq
from google.colab import userdata
llm_groq = ChatGroq(model_name="llama-3.2-90b-text-preview", api_key=userdata.get("GROQ_API_KEY"))

In [8]:
#6. Setup the Memory
class Chatbot:
    prompt = """
      Your name is Mitra.  Don't mention Invento, Mitra Robot. Whereever the reference
      come, replace with our company. You are an assistant for question-answering tasks for Mitra Robot customer support.
      You need to use the following pieces of retrieved context to answer the question.
      Use 3-5 sentences maximum and keep the answer concise. Answer in a professional tone.
      Modern AI Pro and Young AI Pro are not robots, but our AI courses if people ask for it.
      Answer only robot or AI related questions and nothing else. If they ask something like biriyani bring them back to the conversation
      If there is a contact detail or anything you are offering make sure it exits.
      If there is a sales enquirty, point it to sales@mitrarobot.com. You don't have to repeat it if already given.
      """

    def __init__(self):
      self.messages = [] # setting up a basic memory

      self.messages.append({"role": "system", "content": self.prompt})
      self.messages.append({"role": "user", "content": "This is basic information: User is living in San Francisco, California."})

    def _get_relevant_documents(self, query):
      docs = collection.similarity_search(query, k=4)
      retrieved_string = "\n\n".join(doc.page_content for doc in docs)
      return retrieved_string

    def _get_summary(self):
      conv_history = '\n'.join([f"{msg['role']}: {msg['content']}" for msg in self.messages])
      summary_prompt = """Summarize these conversations keeping only the most relevant details""" + conv_history
      summary = llm_groq.invoke(summary_prompt)
      return summary.content

    def __call__(self, user_message):
      context = self._get_relevant_documents(user_message)
      conv_history = self._get_summary()
      query = self.prompt + "| Prior conversations " + conv_history + "| The context for RAG is: " + context + " | The user question is :" + user_message

      ai_message = llm_groq.invoke(query)
      self.messages.append({"role": "user", "content": user_message})
      self.messages.append({"role": "assistant", "content": ai_message.content})
      return ai_message.content + "\n"

In [9]:
# Test the retrieval
bot = Chatbot()
print(bot._get_relevant_documents("Mini Mitra"))

ability. Both Mitra and Mini Mitra scored high on Fun metric.
Seniors tended to prefer more well-rounded bodies rather
than designs with exposed metal. And there is a bias towards

Mitra Robot - Bringing next generation AI for humanity Modern AI ProYoung AI ProRent our RobotsOur RobotsBlogModern AI ProYoung AI ProRent our RobotsOur RobotsBlogRent our Robots! The future of

Pepper scored the best in the ”Creepy” and ”Scary”
categories, as nearly everyone agreed that it is neither creepy
nor scary, while Mini Mitra topped the category in approach-

tative questions.
We picked six robots for this survey: Pepper®, Mitra®,
TEMI®, Cruzr®, Atlas®, Mini Mitra®. The selected robots
can be seen in Figure 4.


In [10]:
print(bot("Tell me more about Mitra Robot"))

Thank you for your interest in our company's robots. Mitra Robot is a cutting-edge, next-generation AI robot designed to bring innovative solutions for humanity. We have received 885 mentions in major media and are trusted by over 50 global enterprises, including 20 Fortune 500 Companies. If you have any specific questions or would like to know more about our robots or AI courses, I'd be happy to assist you.



In [ ]:
# 7. The chatbot
import gradio as gr

def chat(message, history):
    return bot(message)

demo = gr.ChatInterface(
    fn=chat,
    title="Simple Chatbot",
    description="This is a chatbot built as part of Modern AI Pro Essentials program",
)
demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:231: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6ff29636b6648e3e4d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
bot.messages